In [1]:
import nltk
from nltk.corpus import cess_esp
from nltk.tag import UnigramTagger, brill, brill_trainer, CRFTagger
from math import sqrt 
corpus_sentences = cess_esp.tagged_sents()
#Transformamos el corpus para reducir categorias como en la practica 2
corpus_transformado = []
corpus_original = []
for sentence in corpus_sentences:
    sentence_aux = []
    for word, label in sentence:
        if(word != "*0*"):
            if(label.startswith("v") or label.startswith("F")):
                sentence_aux.append((word, label[0:3]))
            else:
                sentence_aux.append((word, label[0:2]))
    corpus_transformado.append(sentence_aux)  

In [2]:
help(nltk.tag.brill)

Help on module nltk.tag.brill in nltk.tag:

NAME
    nltk.tag.brill

DESCRIPTION
    # -*- coding: utf-8 -*-
    # Natural Language Toolkit: Transformation-based learning
    #
    # Copyright (C) 2001-2020 NLTK Project
    # Author: Marcus Uneson <marcus.uneson@gmail.com>
    #   based on previous (nltk2) version by
    #   Christopher Maloof, Edward Loper, Steven Bird
    # URL: <http://nltk.org/>
    # For license information, see  LICENSE.TXT

CLASSES
    nltk.tag.api.TaggerI(builtins.object)
        BrillTagger
    nltk.tbl.feature.Feature(builtins.object)
        Pos
        Word
    
    class BrillTagger(nltk.tag.api.TaggerI)
     |  BrillTagger(initial_tagger, rules, training_stats=None)
     |  
     |  Brill's transformational rule-based tagger.  Brill taggers use an
     |  initial tagger (such as ``tag.DefaultTagger``) to assign an initial
     |  tag sequence to a text; and then apply an ordered list of
     |  transformational rules to correct the tags of individual toke

In [3]:
help(nltk.tag.brill_trainer)

Help on module nltk.tag.brill_trainer in nltk.tag:

NAME
    nltk.tag.brill_trainer

DESCRIPTION
    # -*- coding: utf-8 -*-
    # Natural Language Toolkit: Transformation-based learning
    #
    # Copyright (C) 2001-2013 NLTK Project
    # Author: Marcus Uneson <marcus.uneson@gmail.com>
    #   based on previous (nltk2) version by
    #   Christopher Maloof, Edward Loper, Steven Bird
    # URL: <http://nltk.org/>
    # For license information, see  LICENSE.TXT

CLASSES
    builtins.object
        BrillTaggerTrainer
    
    class BrillTaggerTrainer(builtins.object)
     |  BrillTaggerTrainer(initial_tagger, templates, trace=0, deterministic=None, ruleformat='str')
     |  
     |  A trainer for tbl taggers.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, initial_tagger, templates, trace=0, deterministic=None, ruleformat='str')
     |      Construct a Brill tagger from a baseline tagger and a
     |      set of templates
     |      
     |      :param initial_t

In [4]:
#Creamos diez particiones del corpus tranformado despúes de barajarlo para realizar Cross-Validation
from random import shuffle 
corpus_barajado = corpus_transformado
shuffle(corpus_barajado)
longitud_particion = len(corpus_barajado)//10
particionTest = corpus_barajado[0:longitud_particion]
particionTrain = corpus_barajado[longitud_particion:]
palabrasTest = 0
for sentence in particionTest:
    for word in sentence:
        palabrasTest += 1   

In [5]:
unigram = UnigramTagger(particionTrain)
brill.Template._cleartemplates()
templates = brill.fntbl37()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=unigram, templates = templates)
tagger1 = brillTrainer.train(particionTrain,max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

0.9026496459987127 0.004255150251665331


In [6]:
unigram = UnigramTagger(particionTrain)
brill.Template._cleartemplates()
templates = brill.brill24()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=unigram, templates = templates, trace=1)
tagger1 = brillTrainer.train(particionTrain, max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

TBL train (fast) (seqs: 5427; tokens: 170159; tpls: 24; min score: 2; min acc: None)
Finding initial useful rules...
    Found 55489 useful rules.
Selecting rules...
0.9023814632053208 0.004260374276316841


In [7]:
unigram = UnigramTagger(particionTrain)
brill.Template._cleartemplates()
templates = brill.nltkdemo18plus()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=unigram, templates = templates, trace=1)
tagger1 = brillTrainer.train(particionTrain, max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

TBL train (fast) (seqs: 5427; tokens: 170159; tpls: 23; min score: 2; min acc: None)
Finding initial useful rules...
    Found 70517 useful rules.
Selecting rules...
0.9018450976185368 0.004270792745748961


In [8]:
from nltk import hmm
tagger_hmm = hmm.HiddenMarkovModelTagger.train(particionTrain)
brill.Template._cleartemplates()
templates = brill.fntbl37()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=tagger_hmm, templates = templates, trace=1)
tagger1 = brillTrainer.train(particionTrain, max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

TBL train (fast) (seqs: 5427; tokens: 170159; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 131188 useful rules.
Selecting rules...
0.9342415790602875 0.0035578841947357734


In [9]:
tagger_hmm = hmm.HiddenMarkovModelTagger.train(particionTrain)
brill.Template._cleartemplates()
templates = brill.brill24()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=tagger_hmm, templates = templates, trace=1)
tagger1 = brillTrainer.train(particionTrain, max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

TBL train (fast) (seqs: 5427; tokens: 170159; tpls: 24; min score: 2; min acc: None)
Finding initial useful rules...
    Found 76888 useful rules.
Selecting rules...
0.9342952156189659 0.0035565349748723787


In [10]:
tagger_hmm = hmm.HiddenMarkovModelTagger.train(particionTrain)
brill.Template._cleartemplates()
templates = brill.nltkdemo18plus()
brillTrainer = nltk.tag.brill_trainer.BrillTaggerTrainer(initial_tagger=tagger_hmm, templates = templates, trace=1)
tagger1 = brillTrainer.train(particionTrain, max_rules = 200)
precision = tagger1.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

TBL train (fast) (seqs: 5427; tokens: 170159; tpls: 23; min score: 2; min acc: None)
Finding initial useful rules...
    Found 82776 useful rules.
Selecting rules...
0.9338124865908604 0.0035686536099966836


In [11]:
from nltk.tag.perceptron import PerceptronTagger
pt = PerceptronTagger(load=False)
pt.train(particionTrain)
precision = pt.evaluate(particionTest)
s = sqrt(precision*(1-precision)/palabrasTest) 
intervalo = 1.96*s 
print(precision,intervalo)

0.970553529285561 0.0024266821517585366
